Here's a step-by-step implementation of a neural network for text classification using TensorFlow/Keras:

Step 1: Install required libraries
!pip install tensorflow nltk pandas numpy matplotlib

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Dropout, GlobalMaxPooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

import re
import string

Step 2: Download NLTK resources

In [3]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

Step 3: Load and prepare dataset
We'll use the IMDB movie reviews dataset as an example

In [4]:
# We'll use the IMDB movie reviews dataset as an example
from tensorflow.keras.datasets import imdb

# Load the dataset
vocab_size = 10000  # Keep the top 10,000 most frequent words
max_len = 200  # Maximum length of sequences

(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=vocab_size)

# Pad sequences to ensure uniform length
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)


17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Step 4: Build the neural network model

In [5]:
model = Sequential()

# Add an embedding layer
model.add(Embedding(input_dim=vocab_size, output_dim=128, input_length=max_len))

# Add LSTM layer
model.add(LSTM(64, return_sequences=True))
model.add(Dropout(0.5))

# Add pooling layer
model.add(GlobalMaxPooling1D())

# Add dense layers
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))  # Binary classification

# Compile the model
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Display model architecture
model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d            │ ?                      │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Step 5: Train the model

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history = model.fit(
    X_train, y_train,
    epochs=10,
    batch_size=64,
    validation_split=0.2,
    callbacks=[early_stop],
    verbose=1
)

Epoch 1/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 103s 320ms/step - accuracy: 0.6423 - loss: 0.6054 - val_accuracy: 0.8638 - val_loss: 0.3679
Epoch 2/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 131s 286ms/step - accuracy: 0.9068 - loss: 0.2510 - val_accuracy: 0.8782 - val_loss: 0.3350
Epoch 3/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 88s 281ms/step - accuracy: 0.9436 - loss: 0.1591 - val_accuracy: 0.8738 - val_loss: 0.3062
Epoch 4/10
110/313 ━━━━━━━━━━━━━━━━━━━━ 55s 271ms/step - accuracy: 0.9695 - loss: 0.1015

Step 6: Evaluate the model

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f'Test Accuracy: {accuracy:.4f}')

Step 7: Plot training history

In [ ]:
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

Step 8: Create a function to predict sentiment on new text

In [ ]:
# Get the word index from the IMDB dataset
word_index = imdb.get_word_index()

# Reverse the word index to map back from integers to words
reverse_word_index = {value: key for key, value in word_index.items()}

def decode_review(encoded_review):
    return ' '.join([reverse_word_index.get(i - 3, '?') for i in encoded_review])

def predict_sentiment(text):
    # Preprocess the text
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)

    # Tokenize and convert to sequence
    words = text.split()
    sequence = [word_index[word] if word in word_index and word_index[word] < vocab_size else 2 for word in words]

    # Pad the sequence
    padded_sequence = pad_sequences([sequence], maxlen=max_len)

    # Make prediction
    prediction = model.predict(padded_sequence, verbose=0)[0][0]

    return "Positive" if prediction > 0.5 else "Negative", prediction

Step 9: Test with custom reviews

In [ ]:
test_reviews = [
    "This movie was absolutely fantastic! The acting was superb and the storyline was engaging.",
    "I hated this movie. It was boring and the acting was terrible.",
    "The film was okay, nothing special but not bad either.",
    "This is one of the worst movies I've ever seen. Complete waste of time."
]

for review in test_reviews:
    sentiment, confidence = predict_sentiment(review)
    print(f"Review: {review}")
    print(f"Sentiment: {sentiment} (Confidence: {confidence:.4f})\n")

**Step-by-Step Explanation:**

1. Import Libraries: Import all necessary libraries including TensorFlow, NLTK, and others.

2. Download NLTK Resources: Download required NLTK datasets for text processing.

3. Load Dataset: Use the IMDB movie reviews dataset which is included in Keras.

4. Preprocess Data:

* Convert text to sequences of integers
* Pad sequences to ensure uniform length


5. Build Neural Network:

* Embedding Layer: Converts word indices to dense vectors

* LSTM Layer: Captures sequential patterns in the text

* Pooling Layer: Reduces dimensionality

* Dense Layers: For final classification

6. Train Model:

* Use EarlyStopping to prevent overfitting

* Train for multiple epochs

7. Evaluate Model:

* Calculate accuracy on test set

* Plot training history to visualize performance

8. Create Prediction Function:

* Decode the word index

* Preprocess new text

* Make predictions on custom reviews

9. Test with Custom Text:

* Test the model with sample reviews

* Display sentiment and confidence score